In [1]:
# Importing Libraries
import os, json, random
from collections import Counter
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix, classification_report

C:\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.

In [2]:
# Set seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
# File Paths
DATA_ROOT = Path(r"C:\Users\Thinura\Desktop\safelink\Model\Accident Severity")
WORK_DIR = DATA_ROOT.parent / "accident_severity_supervised_mixup_v3"
WORK_DIR.mkdir(exist_ok=True)


In [4]:
# Model & Training Settings 
IMG_SIZE    = (300, 300)
BATCH       = 32
NUM_CLASSES = 3

In [5]:
# Training Techniques
USE_MIXUP    = True 
LABEL_SMOOTH = 0.05
HEAVY_AUG    = True  
UNFREEZE_TOP = 40    

In [6]:
# Load and Lock Class Names
label_mode = "categorical" if USE_MIXUP else "int"
train_raw = keras.utils.image_dataset_from_directory(
    DATA_ROOT/"train",
    labels="inferred",
    label_mode=label_mode,
    image_size=IMG_SIZE,
    batch_size=BATCH,
)
CLASS_NAMES = train_raw.class_names
print("Locked class order:", CLASS_NAMES)

# Save the class names to a file
with open(WORK_DIR/"class_names.json", "w") as f:
    json.dump(CLASS_NAMES, f)

Found 969 files belonging to 3 classes.
Locked class order: ['HIGH', 'LOW', 'MEDIUM']


In [7]:
# Create Data Augmentation Pipeline
NORM = layers.Rescaling(1./255)
AUG  = keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.35),
    layers.RandomZoom(0.40),
    layers.RandomTranslation(0.30, 0.30),
    layers.RandomContrast(0.40),
    layers.RandomBrightness(0.40),
], name="heavy_aug")

In [8]:
# Define Dataset Creation Function
def make_ds(split: str, augment: bool, shuffle=True):
    ds = keras.utils.image_dataset_from_directory(
        DATA_ROOT/split,
        labels="inferred",
        class_names=CLASS_NAMES,
        label_mode=label_mode,
        image_size=IMG_SIZE,
        batch_size=BATCH,
        shuffle=shuffle,
    )
    def _pp(x, y):
        x = NORM(x)
        if augment and HEAVY_AUG:
            x = AUG(x)
        return x, y
    return ds.map(_pp, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

In [9]:
# Create Train, Validation, and Test Datasets
train_ds = make_ds("train", augment=True,  shuffle=True)
val_ds   = make_ds("valid", augment=False, shuffle=False)
test_ds  = make_ds("test",  augment=False, shuffle=False)

Found 969 files belonging to 3 classes.
Found 207 files belonging to 3 classes.
Found 965 files belonging to 3 classes.


In [10]:
# Define and Apply MixUp Function
def to_one_hot_if_needed(y):
    return y if (len(y.shape) == 2 and y.shape[-1] == NUM_CLASSES) else tf.one_hot(y, depth=NUM_CLASSES)

def mixup_fn(x, y, alpha=0.2):
    y = to_one_hot_if_needed(y)
    lam = tf.constant(np.random.beta(alpha, alpha), dtype=tf.float32)
    bs  = tf.shape(x)[0]
    idx = tf.random.shuffle(tf.range(bs))
    x2, y2 = tf.gather(x, idx), tf.gather(y, idx)
    xm = lam * x + (1. - lam) * x2
    ym = lam * y + (1. - lam) * y2
    return xm, ym

if USE_MIXUP:
    train_ds = train_ds.map(mixup_fn, num_parallel_calls=tf.data.AUTOTUNE)
    loss_fn  = keras.losses.CategoricalCrossentropy(label_smoothing=LABEL_SMOOTH)
else:
    loss_fn  = keras.losses.SparseCategoricalCrossentropy()

metrics = ["accuracy"]

## Model Evaluation

In [11]:
# Build the Model
# Load pre-trained base model
base = keras.applications.EfficientNetV2B2(
    include_top=False, weights="imagenet", input_shape=IMG_SIZE+(3,)
)
base.trainable = False

In [12]:
# Build the full model
inp = keras.Input(shape=IMG_SIZE+(3,))
x   = base(inp, training=False)
x   = layers.GlobalAveragePooling2D()(x)
x   = layers.Dropout(0.5)(x)
out = layers.Dense(NUM_CLASSES, activation="softmax")(x)
model = keras.Model(inp, out)

In [13]:
# Transfer Learning
model.compile(optimizer=keras.optimizers.Adam(3e-4), loss=loss_fn, metrics=metrics)

# Callbacks to help during training
ckpt = keras.callbacks.ModelCheckpoint(str(WORK_DIR/"best_effv2b2_mixup.keras"), save_best_only=True, monitor="val_accuracy")
es   = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, monitor="val_accuracy")
rlr  = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=4, min_lr=1e-6)

print("--- Stage 1: Training with frozen base ---")
model.fit(train_ds, validation_data=val_ds, epochs=40, callbacks=[ckpt, es, rlr])

--- Stage 1: Training with frozen base ---
Epoch 1/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 36s 802ms/step - accuracy: 0.7327 - loss: 0.8234 - val_accuracy: 0.7681 - val_loss: 0.7586 - learning_rate: 3.0000e-04
Epoch 2/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 23s 687ms/step - accuracy: 0.7864 - loss: 0.7285 - val_accuracy: 0.7681 - val_loss: 0.7503 - learning_rate: 3.0000e-04
Epoch 3/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 23s 696ms/step - accuracy: 0.7926 - loss: 0.7265 - val_accuracy: 0.7681 - val_loss: 0.7495 - learning_rate: 3.0000e-04
Epoch 4/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 23s 674ms/step - accuracy: 0.7915 - loss: 0.7141 - val_accuracy: 0.7681 - val_loss: 0.7512 - learning_rate: 3.0000e-04
Epoch 5/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 23s 679ms/step - accuracy: 0.7915 - loss: 0.7210 - val_accuracy: 0.7681 - val_loss: 0.7472 - learning_rate: 3.0000e-04
Epoch 6/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 23s 687ms/step - accuracy: 0.7915 - loss: 0.7149 - val_accuracy: 0.7681 - val_loss: 0.7520 - learning_rate: 3.0000e-04
Epoch 7/40
31/3

In [14]:
# Fine-Tuning
base.trainable = True
for layer in base.layers[:-UNFREEZE_TOP]:
    layer.trainable = False

# Re-compile with a lower learning
model.compile(optimizer=keras.optimizers.Adam(1e-5), loss=loss_fn, metrics=metrics)

print("--- Stage 2: Fine-tuning the top layers ---")
model.fit(train_ds, validation_data=val_ds, epochs=60, callbacks=[ckpt, es, rlr])

--- Stage 2: Fine-tuning the top layers ---
Epoch 1/60
31/31 ━━━━━━━━━━━━━━━━━━━━ 43s 950ms/step - accuracy: 0.6285 - loss: 1.0027 - val_accuracy: 0.7681 - val_loss: 0.7774 - learning_rate: 1.0000e-05
Epoch 2/60
31/31 ━━━━━━━━━━━━━━━━━━━━ 29s 865ms/step - accuracy: 0.7410 - loss: 0.8265 - val_accuracy: 0.7681 - val_loss: 0.7440 - learning_rate: 1.0000e-05
Epoch 3/60
31/31 ━━━━━━━━━━━━━━━━━━━━ 28s 849ms/step - accuracy: 0.7668 - loss: 0.7726 - val_accuracy: 0.7681 - val_loss: 0.7450 - learning_rate: 1.0000e-05
Epoch 4/60
31/31 ━━━━━━━━━━━━━━━━━━━━ 28s 837ms/step - accuracy: 0.7802 - loss: 0.7447 - val_accuracy: 0.7681 - val_loss: 0.7477 - learning_rate: 1.0000e-05
Epoch 5/60
31/31 ━━━━━━━━━━━━━━━━━━━━ 28s 849ms/step - accuracy: 0.7864 - loss: 0.7236 - val_accuracy: 0.7681 - val_loss: 0.7532 - learning_rate: 1.0000e-05
Epoch 6/60
31/31 ━━━━━━━━━━━━━━━━━━━━ 28s 854ms/step - accuracy: 0.7853 - loss: 0.7298 - val_accuracy: 0.7681 - val_loss: 0.7527 - learning_rate: 1.0000e-05
Epoch 7/60
31/

In [15]:
# Evaluate and Save the Model
print("--- Evaluating on the TEST dataset ---")
test_metrics = model.evaluate(test_ds, return_dict=True)
print("TEST Results:", test_metrics)

KERAS_PATH = WORK_DIR/"accident_severity_effv2b2_mixup.keras"
model.save(KERAS_PATH)
print("Saved Keras model to:", KERAS_PATH)

--- Evaluating on the TEST dataset ---
31/31 ━━━━━━━━━━━━━━━━━━━━ 17s 549ms/step - accuracy: 0.7959 - loss: 0.7448
TEST Results: {'accuracy': 0.795854926109314, 'loss': 0.7448330521583557}
Saved Keras model to: C:\Users\Thinura\Desktop\safelink\Model\accident_severity_supervised_mixup_v3\accident_severity_effv2b2_mixup.keras


In [16]:
# Export to TFLite Models

# FP32 version
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_fp32 = converter.convert()
(WORK_DIR/"accident_severity_fp32.tflite").write_bytes(tflite_fp32)

# INT8 dynamic-range version
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_int8 = converter.convert()
(WORK_DIR/"accident_severity_int8.tflite").write_bytes(tflite_int8)

print("TFLite models saved in:", WORK_DIR)

INFO:tensorflow:Assets written to: C:\Users\Thinura\AppData\Local\Temp\tmp2sk0auk8\assets


INFO:tensorflow:Assets written to: C:\Users\Thinura\AppData\Local\Temp\tmp2sk0auk8\assets


Saved artifact at 'C:\Users\Thinura\AppData\Local\Temp\tmp2sk0auk8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 300, 300, 3), dtype=tf.float32, name='keras_tensor_356')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2206998265296: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  2206998264720: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  2206998265680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2207025644176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2207025643984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2206998263760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2207025644560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2207025645328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2207025645712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2207025645520: TensorSpec(shape=(), dtype=tf.reso

INFO:tensorflow:Assets written to: C:\Users\Thinura\AppData\Local\Temp\tmp7tp1cmz9\assets


Saved artifact at 'C:\Users\Thinura\AppData\Local\Temp\tmp7tp1cmz9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 300, 300, 3), dtype=tf.float32, name='keras_tensor_356')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2206998265296: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  2206998264720: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  2206998265680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2207025644176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2207025643984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2206998263760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2207025644560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2207025645328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2207025645712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2207025645520: TensorSpec(shape=(), dtype=tf.reso

In [17]:
# Analyze Performance
y_true, y_pred = [], []
for xb, yb in test_ds:
    probs = model.predict(xb, verbose=0)
    y_pred.extend(np.argmax(probs, axis=1))
    if len(yb.shape) == 2:  # Handle one-hot labels
        yb = tf.argmax(yb, axis=1)
    y_true.extend(yb.numpy())

print("--- Confusion Matrix ---")
print(confusion_matrix(y_true, y_pred))
print("\n--- Classification Report ---")
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES))

--- Confusion Matrix ---
[[768   0   0]
 [ 50   0   0]
 [147   0   0]]

--- Classification Report ---
              precision    recall  f1-score   support

        HIGH       0.80      1.00      0.89       768
         LOW       0.00      0.00      0.00        50
      MEDIUM       0.00      0.00      0.00       147

    accuracy                           0.80       965
   macro avg       0.27      0.33      0.30       965
weighted avg       0.63      0.80      0.71       965



C:\Users\Thinura\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Thinura\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Thinura\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

In [18]:
# Predict on Test Images and Save to CSV
def predict_folder_to_csv(folder: Path, csv_path: Path):
    rows = []
    infer = keras.Sequential([layers.Rescaling(1./255), model])
    exts = {".jpg",".jpeg",".png",".bmp",".webp"}

    for root, _, files in os.walk(folder):
        for fn in files:
            if os.path.splitext(fn)[1].lower() not in exts: continue
            p = Path(root)/fn
            im = Image.open(p).convert("RGB").resize(IMG_SIZE)
            x  = np.asarray(im, dtype=np.float32)[None, ...]
            probs = infer.predict(x, verbose=0)[0]
            idx   = int(np.argmax(probs))
            rows.append({
                "path": str(p),
                "pred": CLASS_NAMES[idx],
                "conf": float(probs[idx]),
                **{f"p_{CLASS_NAMES[i]}": float(probs[i]) for i in range(NUM_CLASSES)}
            })
    pd.DataFrame(rows).to_csv(csv_path, index=False)
    print("Predictions written to:", csv_path)

predict_folder_to_csv(DATA_ROOT/"test", WORK_DIR/"test_predictions.csv")

Predictions written to: C:\Users\Thinura\Desktop\safelink\Model\accident_severity_supervised_mixup_v3\test_predictions.csv


In [2]:
    from pathlib import Path
    Path().cwd()

WindowsPath('C:/Users/Thinura/Final Project')